In [2]:
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate
from langchain.chains.openai_functions import create_structured_output_chain

load_dotenv()
llm = ChatOpenAI(temperature=0.0, model='gpt-3.5-turbo')

In [4]:
planning_template = '''
You are a Software Engineering Manager, responsible for planning the actions of an AI Software Engineer.
You are trying to create the following software:
```
{goal}
```
Here are all the files in the project so far:
[no files yet]

The report from the last time the project was run:
[project has not yet been run]

Plan which file to write to (an existing file, or a new one) and what needs to be done in that file.
After this is done, the program will be run. Plan what to check for and take notes on when the program is being run.
'''[1:-1]

planning_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(planning_template)
])

planning_chain = create_structured_output_chain({
    'type': 'object',
    'properties': {
        'write_filepath': {
            'type': 'string',
            'description': 'Relative path to the file that the engineer should create or edit'
        },
        'write_plan': {
            'type': 'string',
            'description': 'What the engineer should do with the specified file'
        },
        'run_plan': {
            'type': 'string',
            'description': 'What the engineer should look for, or take notes on, when running the program'
        },

        # 'contents': {
        #     'type': 'string',
        #     'description': 'Contents to write to the file.'
        # }

        

    },
    'required': ['write_filepath', 'write_plan', 'run_plan']
}, llm, planning_prompt, verbose=True)#write_template

goal = 'Write a program that finds the first 100 prime numbers.'

plan = planning_chain.run(goal=goal)
print(plan)

DEBUG, CONVERTED FUNCTIONS: [{'name': 'output_formatter', 'description': 'Output formatter. Should always be used to format your response to the user.', 'parameters': {'type': 'object', 'properties': {'write_filepath': {'type': 'string', 'description': 'Relative path to the file that the engineer should create or edit'}, 'write_plan': {'type': 'string', 'description': 'What the engineer should do with the specified file'}, 'run_plan': {'type': 'string', 'description': 'What the engineer should look for, or take notes on, when running the program'}}, 'required': ['write_filepath', 'write_plan', 'run_plan']}}]


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a Software Engineering Manager, responsible for planning the actions of an AI Software Engineer.
You are trying to create the following software:
```
Write a program that finds the first 100 prime numbers.
```
Here are all the files in the project so far:
[no files yet]

The report from the last time the pr

In [7]:
engineering_template = '''
You are a Software Engineer, responsible for executing planned actions.
You are trying to create the following software:
```
{goal}
```

The Engineering Manager AI has planned for you to create the file {filepath}, which does not yet exist. In this file, you are to:
```
{plan}
```
'''[1:-1]
# could dynamically say edit/create based on file existence

engineering_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(engineering_template)
])

engineering_chain = create_structured_output_chain({
    'type': 'object',
    'properties': {
        'file_content': {
            'type': 'string',
            'description': 'Content to write to the given file'
        }
    },
    'required': ['write_filepath', 'write_plan', 'run_plan']
}, llm, engineering_prompt, verbose=True)#write_template

action = engineering_chain.run(goal=goal, plan=plan['write_plan'], filepath=plan['write_filepath'])
print(action)

DEBUG, CONVERTED FUNCTIONS: [{'name': 'output_formatter', 'description': 'Output formatter. Should always be used to format your response to the user.', 'parameters': {'type': 'object', 'properties': {'file_content': {'type': 'string', 'description': 'Content to write to the given file'}}, 'required': ['write_filepath', 'write_plan', 'run_plan']}}]


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a Software Engineer, responsible for executing planned actions.
You are trying to create the following software:
```
Write a program that finds the first 100 prime numbers.
```

The Engineering Manager AI has planned for you to create the file prime_numbers.py, which does not yet exist. You are to:
```
Write a program that finds the first 100 prime numbers.
```

> Finished chain.
{'file_content': "def is_prime(n):\n    if n <= 1:\n        return False\n    for i in range(2, int(n**0.5) + 1):\n        if n % i == 0:\n            return False\n    return True\n\n\ndef 